In [1]:
import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import generate_re_ortho_space, generate_reduced_H, generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_10/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_10/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

10
(10, 27)
(27, 27)
rl errV: 0.00023694902148543046
abs errV: 0.0007546522262828301
errt: 0.0008451096565123393
errh: 0.00013317184525868722
errht: 0.00024513110698447764


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
# #load Hamiltonian generated by above coefficients
#H_correct = scipy.io.mmread('/.mtx').tocsr()
# e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
# e_ground = e1

# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [4]:
#generate thc_mpo
t_spin = get_t_spin(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t_spin)

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [5]:
h1_spin = get_h1_spin(h1)
g_spin = get_g_spin(eri)
g_spin_phy =  g_spin.transpose(0, 2, 1, 3)
mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
print(mpo_ref.bond_dims)

[1, 4, 16, 39, 70, 87, 108, 133, 162, 195, 232, 195, 162, 133, 108, 87, 70, 39, 16, 4, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
hartree_state = generate_single_state(20, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
hartree_state_vector = hartree_state.as_vector()

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [7]:
N_Krylov = 30
psi_original = copy.deepcopy(hartree_state)
#max_bond_Krylov = 40
max_bond_Krylov = 250
trunc_tol = 0
foldername = f"/work_fast/ge49cag/code_datas/Krylov_H10"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:

generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)
#[1, 2, 4, 8, 16, 32, 63, 112, 176, 200, 200, 200, 180, 116, 63, 32, 16, 8, 4, 2, 1]

[1, 2, 4, 7, 11, 16, 22, 29, 37, 46, 56, 46, 37, 29, 22, 16, 11, 7, 4, 2, 1]
2
[1, 2, 4, 8, 16, 31, 57, 99, 163, 204, 250, 204, 163, 99, 57, 31, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 63, 105, 169, 216, 250, 216, 169, 105, 63, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 63, 105, 169, 219, 250, 219, 169, 105, 63, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 62, 104, 168, 219, 250, 219, 168, 104, 62, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 62, 104, 168, 219, 250, 219, 168, 104, 62, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 63, 106, 170, 226, 250, 226, 170, 106, 63, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 63, 107, 171, 231, 250, 231, 171, 107, 63, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 63, 109, 173, 236, 250, 236, 173, 109, 63, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 63, 109, 173, 237, 250, 237, 173, 109, 63, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 63, 109, 173, 240, 250, 240, 173, 109, 63, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 63, 111, 175, 244, 250, 244, 175, 111, 

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [8]:
N_use = 30
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho = generate_reduced_H_non_ortho(N_use, foldername, mpo_ref)
W = get_W(N_use, foldername)
coeff = get_S(W)
coeff = np.array(coeff)
#coeff = coeff_canonical_orthogonalization(N_use, foldername)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced = np.einsum('ik, kl, jl -> ij', coeff.conj(), H_reduced_non_rotho, coeff)
H_reduced = remain_only_tridiagonal_elements(H_reduced)

/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:193: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc/pytenet/global_krylov_method.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = np.vdot(temp1.as_vector(), temp2.as_vector())


In [9]:
N_list = [0]
e_ground = -12.40719648
mps_start = copy.deepcopy(hartree_state)
error_list = [ptn.operation.operator_inner_product(mps_start, mpo_ref, mps_start) - e_ground]

In [10]:
for N in range(5, N_use+1, 5):
    N_list.append(N)
    H_part = H_reduced[:N, :N]
    e_rotate, v_rotate = np.linalg.eigh(H_part)

    #actually we need to orthogonalize the resulting vector to calculate the energy. A option could be add mps together and orthogonalize it.
    v_rotate_ground_coeff = v_rotate[:,0] 
    temp = v_rotate[:,0]
    temp = coeff[:N, :N].transpose(1,0)@temp
    temp = H_reduced_non_rotho[:N, :N]@temp
    temp = coeff[:N, :N].conj()@temp
    e_rotate_ground = ((v_rotate[:,0].reshape(1, N)).conj())@temp
    
    print(e_rotate_ground - e_ground)
    print(e_rotate[0] - e_ground)

[4.96156667]
4.961557239601259
[2.74306877]
2.743066737088144
[1.6624587]
1.6624182048495921
[0.66344706]
0.6634382502814677
[0.43961862]
0.4397400702254348
[0.12784007]
0.12888650121257328


construct approx. ground vector (ritz vector) for re-start:

In [11]:
#assume N = N_use is the best solution
C_ritz = ((v_rotate[:,0].reshape(N_use, 1)).transpose(1, 0))@ coeff
C_ritz  = C_ritz.reshape(C_ritz.shape[1],)

filename = foldername + f"/Krylov_vec{0}.pkl"
with open(filename, 'rb') as file:
    ritz_vec = pickle.load(file)
ritz_vec.A[0] = C_ritz[0]* ritz_vec.A[0]
        
for i in range (1, N_use, 1):
    filename = foldername + f"/Krylov_vec{i}.pkl"
    with open(filename, 'rb') as file:
        temp = pickle.load(file)
    temp.A[0] = C_ritz[i]* temp.A[0]
    ritz_vec = ptn.operation.add_mps_and_compress(ritz_vec, temp, 0, 200)

ritz_vec.orthonormalize('right')
e_ritz = ptn.operation.operator_average(ritz_vec, mpo_ref)
print('ritz energy error:', e_ritz - e_ground)


ritz energy error: (0.1272432944381343+0j)


In [12]:
N_continue = 30
psi_original_continue = copy.deepcopy(ritz_vec)
#max_bond_Krylov = 40
max_bond_Krylov = 250
trunc_tol = 0
foldername_continue = f"/work_fast/ge49cag/code_datas/Krylov_H10_restart"
generate_krylov_space_in_disk(N_continue, H_mu_nu_list_spin_layer, psi_original_continue, max_bond_Krylov, trunc_tol, r_thc, foldername_continue)


[1, 2, 4, 8, 16, 32, 64, 126, 214, 250, 250, 250, 220, 127, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 125, 211, 250, 250, 250, 211, 125, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 125, 207, 250, 250, 250, 209, 125, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 126, 207, 250, 250, 250, 207, 125, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 126, 206, 250, 250, 250, 206, 125, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 205, 125, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 208, 126, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 207, 126, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 207, 126, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 209, 127, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 209, 127, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 126, 205, 250, 250, 250, 2

In [13]:
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho_continue = generate_reduced_H_non_ortho(N_continue, foldername_continue, mpo_ref)
W_continue = get_W(N_continue, foldername_continue)
coeff_continue = get_S(W_continue)
coeff_continue = np.array(coeff_continue) 
#coeff = coeff_canonical_orthogonalization(N_use, foldername)
H_reduced_continue = np.einsum('ik, kl, jl -> ij', coeff_continue.conj(), H_reduced_non_rotho_continue, coeff_continue)
H_reduced_continue = remain_only_tridiagonal_elements(H_reduced_continue)

In [14]:
N_continue_list = [0]
continue_start = copy.deepcopy(ritz_vec)
error_continue_list = [ptn.operation.operator_average(continue_start, mpo_ref) - e_ground]

In [15]:
for N in range(5, N_continue+1, 5):
    N_continue_list.append(N)
    H_part_continue = H_reduced_continue[:N, :N]
    e_rotate_continue, v_rotate_continue = np.linalg.eigh(H_part_continue)

    #actually we need to orthogonalize the resulting vector to calculate the energy. A option could be add mps together and orthogonalize it.
    v_rotate_ground_coeff_continue = v_rotate_continue[:,0] 
    temp = v_rotate_continue[:,0]
    temp = coeff_continue[:N, :N].transpose(1,0)@temp
    temp = H_reduced_non_rotho_continue[:N, :N]@temp
    temp = coeff_continue[:N, :N].conj()@temp
    e_rotate_ground_continue = ((v_rotate_continue[:,0].reshape(1, N)).conj())@temp
    error_continue_list.append(e_rotate_ground_continue - e_ground)
    
    print(e_rotate_ground_continue - e_ground)
    print(e_rotate_continue[0] - e_ground)
    #it looks like orthogonal issue
    #上面两个一样吗？
    #必须要加MPS再normalize吗？
    #check how lanczos in numpy do orthogonaliztion. Ask ppl in the chair?

[0.03881693]
0.038571949715908715
[0.00955283]
0.010272387744576505
[0.00563103]
0.006787475082346006
[0.00532878]
0.006560047866415175
[0.0052973]
0.006547920974519172
[0.00527725]
0.006546963771340231


In [16]:
#assume N = N_use is the best solution
C_ritz_continue = ((v_rotate_continue[:,0].reshape(N_continue, 1)).transpose(1, 0))@ coeff_continue
C_ritz_continue  = C_ritz_continue.reshape(C_ritz_continue.shape[1],)

filename = foldername_continue + f"/Krylov_vec{0}.pkl"
with open(filename, 'rb') as file:
    ritz_vec_continue = pickle.load(file)
ritz_vec_continue.A[0] = C_ritz_continue[0]* ritz_vec_continue.A[0]
        
for i in range (1, N_continue, 1):
    filename = foldername_continue + f"/Krylov_vec{i}.pkl"
    with open(filename, 'rb') as file:
        temp = pickle.load(file)
    temp.A[0] = C_ritz_continue[i]* temp.A[0]
    ritz_vec_continue = ptn.operation.add_mps_and_compress(ritz_vec_continue, temp, 0, 200)

ritz_vec_continue.orthonormalize('right')
e_ritz_continue = ptn.operation.operator_average(ritz_vec_continue, mpo_ref)
print('ritz energy error:', e_ritz_continue - e_ground)


ritz energy error: (0.004542828831684886+0j)


In [17]:
N_continue_a = 30
psi_original_continue_a = copy.deepcopy(ritz_vec_continue)
#max_bond_Krylov = 40
max_bond_Krylov = 250
trunc_tol = 0
foldername_continue_a = f"/work_fast/ge49cag/code_datas/Krylov_H10_restart_a"
generate_krylov_space_in_disk(N_continue_a, H_mu_nu_list_spin_layer, psi_original_continue_a, max_bond_Krylov, trunc_tol, r_thc, foldername_continue_a)


[1, 2, 4, 8, 16, 32, 64, 127, 229, 250, 250, 250, 234, 128, 64, 32, 16, 8, 4, 2, 1]
2
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 228, 128, 64, 32, 16, 8, 4, 2, 1]
3
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 226, 128, 64, 32, 16, 8, 4, 2, 1]
4
[1, 2, 4, 8, 16, 32, 64, 127, 224, 250, 250, 250, 228, 128, 64, 32, 16, 8, 4, 2, 1]
5
[1, 2, 4, 8, 16, 32, 64, 127, 227, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
6
[1, 2, 4, 8, 16, 32, 64, 127, 227, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
7
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 227, 128, 64, 32, 16, 8, 4, 2, 1]
8
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
9
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
10
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
11
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 225, 127, 64, 32, 16, 8, 4, 2, 1]
12
[1, 2, 4, 8, 16, 32, 64, 127, 226, 250, 250, 250, 2

In [18]:
H_reduced_non_rotho_continue_a = generate_reduced_H_non_ortho(N_continue_a, foldername_continue_a, mpo_ref)
W_continue_a = get_W(N_continue_a, foldername_continue_a)
coeff_continue_a = get_S(W_continue_a)
coeff_continue_a = np.array(coeff_continue_a) 
#coeff = coeff_canonical_orthogonalization(N_use, foldername)
H_reduced_continue_a = np.einsum('ik, kl, jl -> ij', coeff_continue_a.conj(), H_reduced_non_rotho_continue_a, coeff_continue_a)
H_reduced_continue_a = remain_only_tridiagonal_elements(H_reduced_continue_a)

In [19]:
e_rotate_continue_a, v_rotate_continue_a = np.linalg.eigh(H_reduced_continue_a)



C_ritz_continue_a = ((v_rotate_continue_a[:,0].reshape(N_continue_a, 1)).transpose(1, 0))@ coeff_continue_a
C_ritz_continue_a  = C_ritz_continue_a.reshape(C_ritz_continue_a.shape[1],)

filename = foldername_continue_a + f"/Krylov_vec{0}.pkl"
with open(filename, 'rb') as file:
    ritz_vec_continue_a = pickle.load(file)
ritz_vec_continue_a.A[0] = C_ritz_continue_a[0]* ritz_vec_continue_a.A[0]
        
for i in range (1, N_continue_a, 1):
    filename = foldername_continue_a + f"/Krylov_vec{i}.pkl"
    with open(filename, 'rb') as file:
        temp = pickle.load(file)
    temp.A[0] = C_ritz_continue_a[i]* temp.A[0]
    ritz_vec_continue_a = ptn.operation.add_mps_and_compress(ritz_vec_continue_a, temp, 0, 200)

ritz_vec_continue_a.orthonormalize('right')
e_ritz_continue_a = ptn.operation.operator_average(ritz_vec_continue_a, mpo_ref)
print('ritz energy error:', e_ritz_continue_a - e_ground)


ritz energy error: (0.00013621676716724096+0j)


In [20]:
#To do: 
#max-bond 设置为250
#每步30， 一共restart三次
